# SADS 모델의 Unicast Phase 동작 검증을 위한 데이터 생성

* 공격자의 광고 패킷이 존재한다는 가정 하에 동작 검증
* 공격자의 광고 패킷이 연속적으로 수신되는 가정 하에 동작 검증
* 공격자의 광고 패킷을 스푸핑 공격으로 감지하는 TN (= True Negative) 검증

In [1]:
import csv
import numpy as np
import pandas as pd
import random

In [2]:
# seed = 25632
random.seed("25632")

In [3]:
source_file = './data/original/data1.csv'
write_file = './data/true-negative/data1_tn.csv'

df = pd.read_csv(source_file)
df_time = df['time']

rf = open(source_file, 'r', encoding='utf-8')
data = list(csv.reader(rf))

wf = open(write_file, 'w', encoding='utf-8', newline='')
wr = csv.writer(wf)

In [4]:
print("최초 입력 시간 :", df_time[0])
print("마지막 입력 시간 :", df_time[len(df_time)-1])
print("데이터 갯수 :",len(df_time))
print("총 소요 시간 :", df_time[len(df_time)-1] - df_time[0])

최초 입력 시간 : 1641964485.5144866
마지막 입력 시간 : 1641964899.6425989
데이터 갯수 : 2000
총 소요 시간 : 414.1281123161316


In [5]:
end_time = df_time[len(df_time)-1]
curr_time = df_time[0]

time_list = []
time_list.append(curr_time)

idx = 0

# Time List's Odd Index => Promise Time's End
# Time List's Even Index => Non-promise Time's End
while (curr_time <= end_time):
    if idx % 2 == 0:
        v = random.randint(5, 7)
        time_list.append(curr_time + v)
        curr_time += v
    else:
        v = random.randint(2, 5)
        time_list.append(curr_time + v)
        curr_time += v
        
    idx += 1

print("Time List's Length :", len(time_list))
# print("Time List :", time_list)

Time List's Length : 88


In [6]:
# Spoofing Data Input Index
r_idx = random.randint(1, len(time_list) // 2)

print("Random Index :", r_idx)

Random Index : 18


In [7]:
data_tn = []

for i in range(1,len(data)):
    row = data[i]
    
    # tmp = [time, rssi, isTag]
    tmp = []
    tmp.append(float(row[2]))
    tmp.append(int(row[3]))
    tmp.append(True)
    
    data_tn.append(tmp)

# 사용자와 공격자 사이의 거리가 매우 가까움
mean_rssi = np.mean(df['rssi'])

# time_list[r_idx]부터 Spoofing Data 수신
spoof_time = time_list[r_idx]

# Spoofing Data가 연속적으로 끝까지 수신됨
while spoof_time <= df_time[len(df_time)-1]:
    # tmp = [time, rssi, isTag]
    tmp = []
    tmp.append(spoof_time)
    tmp.append(mean_rssi)
    tmp.append(False)
    
    data_tn.append(tmp)
    
    # 광고 패킷 평균 송신 시간 간격 (= 0.13)
    spoof_time += 0.13
    
data_tn.sort()
header = ["time", "rssi", "isTag"]
data_tn.insert(0,header)

for row in data_tn:
    wr.writerow(row)

wf.close()